In [ ]:
import pathlib
import zipfile
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models
import pandas as pd
import tarfile 
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras

In [ ]:
df = pd.read_csv('fer2013.csv')
df.head()

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
emotion_label_to_text = {0:'anger', 1:'disgust', 2:'fear', 3:'happiness', 4: 'sadness', 5: 'surprise', 6: 'neutral'}

In [ ]:
import numpy as np
def augment_pixels(px, IMG_SIZE = 48):
    image = np.array(px.split(' ')).reshape(IMG_SIZE, IMG_SIZE).astype('float32')
    image = tf.image.random_flip_left_right(image.reshape(IMG_SIZE,IMG_SIZE,1))
    # Pad image size
    image = tf.image.resize_with_crop_or_pad(image, IMG_SIZE + 12, IMG_SIZE + 12) 
    # Random crop back to the original size
    image = tf.image.random_crop(image, size=[IMG_SIZE, IMG_SIZE, 1])
    image = tf.image.random_brightness(image, max_delta=0.5) # Random brightness
    image = tf.clip_by_value(image, 0, 255)
    augmented = image.numpy().reshape(IMG_SIZE,IMG_SIZE)
    str_augmented = ' '.join(augmented.reshape(IMG_SIZE*IMG_SIZE).astype('int').astype(str))
    return str_augmented

valcounts = df.emotion.value_counts()
valcounts_diff = valcounts[valcounts.idxmax()] - valcounts
for emotion_idx, aug_count in valcounts_diff.iteritems():
    sampled = df.query("emotion==@emotion_idx").sample(aug_count, replace=True)
    sampled['pixels'] = sampled.pixels.apply(augment_pixels)
    df = pd.concat([df, sampled])
    print(emotion_idx, aug_count)

3 0
6 2791
4 2912
2 3868
0 4036
5 4987
1 8442


In [ ]:
INTERESTED_LABELS = [0, 1, 2, 3, 4, 5, 6]

In [ ]:
df = df[df.emotion.isin(INTERESTED_LABELS)]
df

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training
...,...,...,...
2787,1,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,Training
20137,1,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,Training
10053,1,0 14 16 21 38 41 55 102 120 124 129 133 134 14...,Training
9366,1,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,Training


In [ ]:
le = LabelEncoder()
img_labels = le.fit_transform(df.emotion)
img_labels = np_utils.to_categorical(img_labels)


In [ ]:
img_array = df.pixels.apply(lambda x: np.array(x.split(' ')).reshape(48, 48, 1).astype('float32'))
img_array = np.stack(img_array, axis=0)

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(img_array, img_labels,
                                                    shuffle=True, stratify=img_labels,
                                                    test_size=0.2, random_state=42)
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

((50338, 48, 48, 1), (12585, 48, 48, 1), (50338, 7), (12585, 7))

In [ ]:
model = models.Sequential([
            
    layers.Conv2D(filters=48,kernel_size=(5,5),  input_shape = (48, 48, 1),activation='relu'),
    layers.MaxPooling2D(pool_size=(2,2)),

    layers.Conv2D(filters=48,kernel_size=(3,3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Dropout(0.25),

    #layers.Conv2D(filters=64,kernel_size=(3,3), activation='relu'),
    #layers.MaxPooling2D(pool_size=(2,2)),
    #layers.Dropout(0.25),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.25),
    layers.Dense(7, activation='softmax')
])

In [ ]:
model.compile(optimizer='adam',
              loss="categorical_crossentropy",
              metrics=['acc'])

In [ ]:
X_train = X_train / 255.
X_valid = X_valid / 255.

In [ ]:
history = model.fit(X_train,y_train,validation_split=0.16, epochs=20)

Epoch 1/20
1322/1322 [==============================] - 168s 127ms/step - loss: 0.8329 - acc: 0.6852 - val_loss: 1.3628 - val_acc: 0.5243
Epoch 2/20
1322/1322 [==============================] - 169s 128ms/step - loss: 0.8237 - acc: 0.6880 - val_loss: 1.3263 - val_acc: 0.5255
Epoch 3/20
1322/1322 [==============================] - 166s 125ms/step - loss: 0.8125 - acc: 0.6920 - val_loss: 1.3370 - val_acc: 0.5273
Epoch 4/20
1322/1322 [==============================] - 161s 122ms/step - loss: 0.8070 - acc: 0.6938 - val_loss: 1.3764 - val_acc: 0.5241
Epoch 5/20
1322/1322 [==============================] - 158s 120ms/step - loss: 0.8016 - acc: 0.6947 - val_loss: 1.3579 - val_acc: 0.5238
Epoch 6/20
1322/1322 [==============================] - 157s 119ms/step - loss: 0.7974 - acc: 0.6966 - val_loss: 1.3840 - val_acc: 0.5295
Epoch 7/20
1322/1322 [==============================] - 158s 120ms/step - loss: 0.7857 - acc: 0.7025 - val_loss: 1.3677 - val_acc: 0.5232
Epoch 8/20
1322/1322 [============